# Implicit Alternating Least Squares

A recommendation model using an alternating least squares factorization approach for implicit datasets.

[Iplicit ALS Documentation](https://implicit.readthedocs.io/en/latest/als.html)

## Table of contents

* [Sample files](#sample-files)
* [Step 1 - Prepare training data](#prepare-training-data)
 * [Download lastfm 360k dataset](#download)
 * [Prepare lastfm artist play training data](#prepare-data)
 * [Create training data file](#create-training-data-file)
 * [Upload training data file](#upload-training-data)
* [Step 2 - Create a model](#create-model)
 * [Run a SageMaker training job](#run-training-job)
 * [Create a SageMaker model](#create-sagemaker-model)
* [Step 3 - Get recommendations (inference)](#get-recommendations)
 * [Example users](#example-users)
 * [Create batch transform input file](#create-batch-input)
 * [Upload the batch transform input file to s3](#upload-batch-input)
 * [Run the Batch Transform Job](#run-transform)
 * [Download the batch results](#download-batch-results)
 * [Recommendations with scores](#recommendations)
 * [User history](#user-history)
* [Step 4 - Optional Cleanup](#cleanup)

## Sample files <a id="sample-files"></a>

These links are to example files on github.

* [training input file](https://github.com/outpace/sagemaker-examples/blob/master/implicit-als/training/lastfm-360k-1mm-clean.csv)
* [batch transform input file](https://github.com/outpace/sagemaker-examples/blob/master/implicit-als/batch_input/recommendation.requests)
* [batch transform output file](https://github.com/outpace/sagemaker-examples/blob/master/implicit-als/recommendation.requests.out)

## Step 1 - Prepare training data <a id="prepare-training-data"></a>
### Download lastfm 360k dataset <a id="download"></a>

In [1]:
!wget --no-clobber http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
!tar -xvf lastfm-dataset-360K.tar.gz
!head lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv

--2018-11-08 17:33:11--  http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
Resolving mtg.upf.edu (mtg.upf.edu)... 84.89.139.55
Connecting to mtg.upf.edu (mtg.upf.edu)|84.89.139.55|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569202935 (543M) [application/x-gzip]
Saving to: ‘lastfm-dataset-360K.tar.gz’

lastfm-dataset-360K 100%[===================>] 542.83M  1.53MB/s    in 5m 56s  

2018-11-08 17:39:08 (1.52 MB/s) - ‘lastfm-dataset-360K.tar.gz’ saved [569202935/569202935]

lastfm-dataset-360K/
lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
lastfm-dataset-360K/README.txt
lastfm-dataset-360K/mbox_sha1sum.py
lastfm-dataset-360K/usersha1-profile.tsv
00000c289a1829a808ac09c00daf10bc3c4e223b	3bd73256-3905-4f3a-97e2-8b341527f805	betty blowtorch	2137
00000c289a1829a808ac09c00daf10bc3c4e223b	f2fb0ff0-5679-42ec-a55c-15109ce6e320	die Ärzte	1099
00000c289a1829a808ac09c00daf10bc3c4e223b	b3ae82c2-e60b-4551-a76d-6620f1b456aa	melissa etheridge	897

### Prepare lastfm artist play training data <a id="prepare-data"></a>

Import the tab separated lastfm file. Take only the first 1 million rows to save memory and processing time. Drop any rows with null values in `item_id`, `user_id`, `total_interactions`, `artist_name`.

In [2]:
import pandas as pd

df = pd.read_csv('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv', 
                 sep="\t", 
                 header=None, 
                 names=["user_id", "item_id", "artist_name", "total_interactions"], 
                 nrows=1000000)
df = df.dropna(subset=['item_id', 'user_id', 'total_interactions', 'artist_name'])
print(df.shape)
df.head()

(987161, 4)


,user_id,item_id,artist_name,total_interactions
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


### Create training data file <a id="create-training-data-file"></a>

Create a csv file from the dataframe above. Do not include the index, but include headers `user_id`, `item_id`, and `total_interactions`. Show the head of the file.

In [3]:
data_dir = 'implicit-als'
train_data_dir = '{}/training'.format(data_dir)
train_data_file = '{}/lastfm-360k-1mm-clean.csv'.format(train_data_dir)

!mkdir -p {train_data_dir}
df[["user_id", "item_id", "total_interactions"]].to_csv(train_data_file, index=False)

!head {train_data_file}

user_id,item_id,total_interactions
00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,2137
00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,1099
00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,897
00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,717
00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,706
00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,691
00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,545
00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,507
00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,424


### Upload training data to s3 <a id="upload-training-data"></a>

Choose a bucket, optionally customize the prefix, and upload the csv created above.

In [4]:
import sagemaker as sage

bucket = "sagemaker-validation-us-east-2"
prefix = "implicit-als-test"

sess = sage.Session()

s3_train = sess.upload_data(train_data_dir, bucket, "{}/training".format(prefix))
"uploaded training data file to {}".format(s3_train)

'uploaded training data file to s3://sagemaker-validation-us-east-2/implicit-als-test/training'

## Step 2 - Create a model <a id="create-model"></a>

### Run a SageMaker training job <a id="run-training-job"></a>

This code will start a training job, wait for it to be done, and report its status.

In [5]:
%%time

import boto3
import time
from sagemaker import get_execution_role

role = get_execution_role()
ecr_image = "435525115971.dkr.ecr.us-east-2.amazonaws.com/sagemaker/implicit-als:9"
job_name_prefix = 'implicit-als-test'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp
create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": ecr_image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, job_name_prefix)
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.p3.2xlarge",
        "VolumeSizeInGB": 50
    },
    "TrainingJobName": job_name,
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 360000
    },
    "InputDataConfig": [
        {
            "ChannelName": "training",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_train,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/csv",
            "CompressionType": "None"
        }
    ]
}

sagemaker = boto3.client(service_name='sagemaker')
sagemaker.create_training_job(**create_training_params)
status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

try:
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    job_info = sagemaker.describe_training_job(TrainingJobName=job_name)
    status = job_info['TrainingJobStatus']
    print("Training job ended with status: " + status)
except:
    print('Training failed to start')
    message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print('Training failed with the following error: {}'.format(message))

Training job current status: InProgress
Training job ended with status: Completed
CPU times: user 167 ms, sys: 9.02 ms, total: 177 ms
Wall time: 4min


### Create a SageMaker model <a id="create-sagemaker-model"></a>

This will set up the model created during training within SageMaker to be used later for recommendations.

In [6]:
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name="implicit-als-test" + timestamp
job_info = sagemaker.describe_training_job(TrainingJobName=job_name)
model_data = job_info['ModelArtifacts']['S3ModelArtifacts']

primary_container = {
    'Image': ecr_image,
    'ModelDataUrl': model_data,
}

create_model_response = sagemaker.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

create_model_response

{'ModelArn': 'arn:aws:sagemaker:us-east-2:435525115971:model/implicit-als-test-2018-11-08-17-43-37',
 'ResponseMetadata': {'RequestId': 'a97c4ffb-fcb2-49b2-adac-719aeaa066da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a97c4ffb-fcb2-49b2-adac-719aeaa066da',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Thu, 08 Nov 2018 17:43:37 GMT'},
  'RetryAttempts': 0}}

## Step 3 - Get recommendations (Inference) <a id="get-recommendations"></a>

### Example users <a id="example-users"></a>

Find some example users in order to predict their next rating/watch.

In [7]:
example_users = df[df.user_id.isin(["05c4bbb936abd2331e8f64037c95a61335d40e30",
                                    "030ebbd1d8b360ce465a20e30a67a43da97f1b20"])]
example_users

,user_id,item_id,artist_name,total_interactions
207869,030ebbd1d8b360ce465a20e30a67a43da97f1b20,a7022764-95fb-46af-a7d6-90056746451a,uma thurman,651
207870,030ebbd1d8b360ce465a20e30a67a43da97f1b20,0743b15a-3c32-48c8-ad58-cb325350befa,blink-182,649
391892,05c4bbb936abd2331e8f64037c95a61335d40e30,99d7b49c-c18e-4a11-bf3e-b71710938df6,phoenix,3
391893,05c4bbb936abd2331e8f64037c95a61335d40e30,bd4d397a-849a-48bf-be24-52eec87feeee,adriana calcanhotto,2


### Create batch transform input file <a id="create-batch-input"></a>

Each row is a json object containing two keys:

* `user_id`: the id of user
* `top_n`: the number of top scoring recommendations to return

The head of the batch input file is shown.

In [8]:
import json

batch_input_dir = '{}/batch_input'.format(data_dir)
batch_input_file = batch_input_dir + '/recommendation.requests'

!mkdir -p {batch_input_dir}

with open(batch_input_file, 'w') as outfile:
    json.dump({"user_id": "05c4bbb936abd2331e8f64037c95a61335d40e30", "top_n": "5"}, outfile)
    outfile.write("\n")
    json.dump({"user_id": "030ebbd1d8b360ce465a20e30a67a43da97f1b20", "top_n": "5"}, outfile)
   
!head {batch_input_file}

{"user_id": "05c4bbb936abd2331e8f64037c95a61335d40e30", "top_n": "5"}
{"user_id": "030ebbd1d8b360ce465a20e30a67a43da97f1b20", "top_n": "5"}

### Upload the batch transform input file to s3 <a id="upload-batch-input"></a>

In [9]:
batch_input = sess.upload_data(batch_input_dir, bucket, "{}/batch_input".format(prefix))
"uploaded training data file to {}".format(batch_input)

'uploaded training data file to s3://sagemaker-validation-us-east-2/implicit-als-test/batch_input'

### Run the Batch Transform Job <a id="run-transform"></a>

This code will start a batch transform job, wait for it to be done, and report its status.

In [10]:
%%time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
batch_job_name = "implicit-als-test" + timestamp
batch_output = 's3://{}/{}/output'.format(bucket, batch_job_name)
request = \
{
  "TransformJobName": batch_job_name,
  "ModelName": model_name,
  "BatchStrategy": "SingleRecord",
  "TransformInput": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": batch_input
      }
    },
    "ContentType": "application/json",
    "CompressionType": "None",
    "SplitType": "Line"
  },
  "TransformOutput": {
    "S3OutputPath": batch_output,
    "Accept": "text/csv",
    "AssembleWith": "Line"
  },
  "TransformResources": {
    "InstanceType": "ml.p3.2xlarge",
    "InstanceCount": 1
  }
}

sagemaker.create_transform_job(**request)

print("Created Transform job with name: ", batch_job_name)

while(True):
    job_info = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = job_info['TransformJobStatus']
    if status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = job_info['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    time.sleep(30)

Created Transform job with name:  implicit-als-test-2018-11-08-17-43-38
Transform job ended with status: Completed
CPU times: user 92 ms, sys: 13.4 ms, total: 105 ms
Wall time: 4min 1s


### Download the batch results <a id="download-batch-results"></a>

Show the head of the file.

In [11]:
!aws s3 cp {batch_output + '/recommendation.requests.out'} {data_dir}

!head {data_dir}/recommendation.requests.out

download: s3://sagemaker-validation-us-east-2/implicit-als-test-2018-11-08-17-43-38/output/recommendation.requests.out to implicit-als/recommendation.requests.out
05c4bbb936abd2331e8f64037c95a61335d40e30,bd4d397a-849a-48bf-be24-52eec87feeee,0.024679340422153473
05c4bbb936abd2331e8f64037c95a61335d40e30,99d7b49c-c18e-4a11-bf3e-b71710938df6,0.0211029015481472
05c4bbb936abd2331e8f64037c95a61335d40e30,f81f19b9-c76e-43ac-8656-bb56071785fb,0.02010958269238472
05c4bbb936abd2331e8f64037c95a61335d40e30,f07dbc2f-317b-470f-bad4-5f1b0eb6faf1,0.0188058540225029
05c4bbb936abd2331e8f64037c95a61335d40e30,557ac15e-7800-44c3-bb62-d4ab34ac1e28,0.017942411825060844
030ebbd1d8b360ce465a20e30a67a43da97f1b20,0743b15a-3c32-48c8-ad58-cb325350befa,0.7281188368797302
030ebbd1d8b360ce465a20e30a67a43da97f1b20,a7022764-95fb-46af-a7d6-90056746451a,0.7025371193885803
030ebbd1d8b360ce465a20e30a67a43da97f1b20,c2a44e93-3a2b-44aa-bd8b-7a71bb76e3b5,0.435314804315567
030ebbd1d8b360ce465a20e30a67a43da97f1b20,f2eef649-a6d5-41

### Recommendations with scores <a id="recommendations"></a>

Import the recommendations from the batch output file downloaded above and join with artist names. These are the top 5 artist recommendations for our example users.

In [12]:
recommendations_df = pd.read_csv('{}/recommendation.requests.out'.format(data_dir), 
                                 header=None, 
                                 names=["user_id", "item_id", "score"])
artist_names = df.groupby(['item_id']).agg(lambda x: x.iloc[0])[["artist_name"]]
recommendations_df = recommendations_df.join(artist_names, on='item_id')
recommendations_df

,user_id,item_id,score,artist_name
0,05c4bbb936abd2331e8f64037c95a61335d40e30,bd4d397a-849a-48bf-be24-52eec87feeee,0.024679,adriana calcanhotto
1,05c4bbb936abd2331e8f64037c95a61335d40e30,99d7b49c-c18e-4a11-bf3e-b71710938df6,0.021103,phoenix
2,05c4bbb936abd2331e8f64037c95a61335d40e30,f81f19b9-c76e-43ac-8656-bb56071785fb,0.020110,marisa monte
3,05c4bbb936abd2331e8f64037c95a61335d40e30,f07dbc2f-317b-470f-bad4-5f1b0eb6faf1,0.018806,caetano veloso
4,05c4bbb936abd2331e8f64037c95a61335d40e30,557ac15e-7800-44c3-bb62-d4ab34ac1e28,0.017942,los hermanos
5,030ebbd1d8b360ce465a20e30a67a43da97f1b20,0743b15a-3c32-48c8-ad58-cb325350befa,0.728119,blink-182
6,030ebbd1d8b360ce465a20e30a67a43da97f1b20,a7022764-95fb-46af-a7d6-90056746451a,0.702537,uma thurman
7,030ebbd1d8b360ce465a20e30a67a43da97f1b20,c2a44e93-3a2b-44aa-bd8b-7a71bb76e3b5,0.435315,+44
8,030ebbd1d8b360ce465a20e30a67a43da97f1b20,f2eef649-a6d5-4114-afba-e50ab26254d2,0.434301,sum 41
9,030ebbd1d8b360ce465a20e30a67a43da97f1b20,67f66c07-6e61-4026-ade5-7e782fad3a5d,0.395946,foo fighters


### User history <a id="user-history"></a>

Show the example users' history again for convenience.

In [13]:
example_users

,user_id,item_id,artist_name,total_interactions
207869,030ebbd1d8b360ce465a20e30a67a43da97f1b20,a7022764-95fb-46af-a7d6-90056746451a,uma thurman,651
207870,030ebbd1d8b360ce465a20e30a67a43da97f1b20,0743b15a-3c32-48c8-ad58-cb325350befa,blink-182,649
391892,05c4bbb936abd2331e8f64037c95a61335d40e30,99d7b49c-c18e-4a11-bf3e-b71710938df6,phoenix,3
391893,05c4bbb936abd2331e8f64037c95a61335d40e30,bd4d397a-849a-48bf-be24-52eec87feeee,adriana calcanhotto,2


## Step 4 - Optional Clean up <a id="cleanup"></a>

In [14]:
def cleanup():
    !rm lastfm-dataset-360K.tar.gz 2> /dev/null
    !rm -fr implicit-als/ 2> /dev/null
    !rm -fr lastfm-dataset-360K/ 2> /dev/null
    sagemaker.delete_model(ModelName = model_name)
    
# optionally uncomment and run the code to clean everything up  

#cleanup()